### Imports

In [7]:
# !pip install -e ../.

In [8]:
# Install proper version of torch, as according to: https://pytorch.org/
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [1]:
import pandas as pd
import torch
from looptune import prep_config_combinations, single_run, clean_memory

### Prepare dataset with two columns: 'text' and 'label'

Examplary data source: https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news


In [2]:
df = pd.read_csv('example_data/SentimentAnalysisforFinancialNews.csv', encoding="ISO-8859-1", header=None)
df.columns = ['label', 'text']
df.head(5)

,label,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


### Prepare run configurations

In [5]:
run_config = {   # -----------------------
                 'model_name': 'meta-llama/Meta-Llama-3-8B', # Pre-trained model names from the Hugging Face hub used for fine-tuning
                 # --------------------------
                  'split': (0.7, 0.3), # Divides the dataset into training, testing, (and optionally) validation sets. Examples: (0.7,0.3) -> split into train and test proportionally; (70, 30) splits into train,test proportionally.
                 'binary': False, # Indicates whether the task is binary (two classes) or multi-class classification.,
                 'balanced': (('train',), ('test',)),
                 # --------------------------
                 'training_arguments': {
                     'num_train_epochs': 1, # Number of times the model sees the entire training dataset.
                     'per_device_train_batch_size': 4, # Number of samples processed in each training step (personally, 8/16 work best, 16 is faster, but you may find linear drop in inference speed during fine-tuning).
                     'per_device_eval_batch_size': 4, # Number of samples processed in each evaluation step.
                     # 'gradient_accumulation_steps': 4,
                     'gradient_checkpointing': True,
                     #-----------------------------
                     'save_total_limit': 2,
                     'load_best_model_at_end': True,
                     'save_strategy': 'steps', # Controls when to save model checkpoints ('steps', 'epoch' or 'no').
                     'metric_for_best_model': 'f1-score',
                     #-----------------------------
                     'evaluation_strategy': "steps",
                     'logging_steps': 100,
                     'fp16': False,
                     # 'use_cpu': False,
                     #-----------------------------
                     'learning_rate': 2e-5,
                     'lr_scheduler_type': "linear",
                     'warmup_ratio': 0.1,
                     'max_grad_norm': 0.3,
                     'weight_decay': 0.001,
                 },
                 #-----------------------------
                     'bnb_config': [
                                # False,
                                {'bnb_4bit_compute_dtype': torch.bfloat16, 'load_in_4bit': True, 'bnb_4bit_quant_type': "nf4", 'bnb_4bit_use_double_quant': True, 'load_in_8bit': False}
                                 ],
                 'peft_config': [
                                # False,
                                {'r': 8, 'lora_alpha': 32, 'lora_dropout': 0.05, 'bias': "none",
                                'task_type': "SEQ_CLS", 
                                # 'target_modules': ("v_proj",),
                                'target_modules': "all-linear"
                                }
                                ],
                    }

run_params_serie = prep_config_combinations(run_config)
run_params_serie

[{'model_name': 'meta-llama/Meta-Llama-3-8B',
  'split': (0.7, 0.3),
  'binary': False,
  'balanced': (('train',), ('test',)),
  'training_arguments': {'num_train_epochs': 1,
   'per_device_train_batch_size': 4,
   'per_device_eval_batch_size': 4,
   'gradient_checkpointing': True,
   'save_total_limit': 2,
   'load_best_model_at_end': True,
   'save_strategy': 'steps',
   'metric_for_best_model': 'f1-score',
   'evaluation_strategy': 'steps',
   'logging_steps': 100,
   'fp16': False,
   'learning_rate': 2e-05,
   'lr_scheduler_type': 'linear',
   'warmup_ratio': 0.1,
   'max_grad_norm': 0.3,
   'weight_decay': 0.001},
  'bnb_config': {'bnb_4bit_compute_dtype': torch.bfloat16,
   'load_in_4bit': True,
   'bnb_4bit_quant_type': 'nf4',
   'bnb_4bit_use_double_quant': True,
   'load_in_8bit': False},
  'peft_config': {'r': 8,
   'lora_alpha': 32,
   'lora_dropout': 0.05,
   'bias': 'none',
   'task_type': 'SEQ_CLS',
   'target_modules': 'all-linear'}}]

In [6]:
for run_params in run_params_serie:
    single_run(run_params, df)

          text
label         
negative   604
neutral   2879
positive  1363


Casting to class labels:   0%|          | 0/4846 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1269 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/543 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

D:\Users\Jakub\anaconda3\envs\looptune\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jakubpart (jpartyka). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
D:\Users\Jakub\anaconda3\envs\looptune\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
D:\Users\Jakub\anaconda3\envs\looptune\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,1.829600,1.210119,0.546508,0.508287,0.501747,0.508287
200,0.998000,0.884843,0.718395,0.686924,0.669811,0.686924
300,0.752400,0.641259,0.754789,0.751381,0.752562,0.751381


D:\Users\Jakub\anaconda3\envs\looptune\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
D:\Users\Jakub\anaconda3\envs\looptune\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
D:\Users\Jakub\anaconda3\envs\looptune\Lib\site-

eval/accuracy,▁▆█
eval/f1-score,▁▆█
eval/loss,█▄▁
eval/precision,▁▇█
eval/recall,▁▆█
eval/runtime,█▁▆
eval/samples_per_second,▁█▃
eval/steps_per_second,▁█▃
train/epoch,▁▁▄▄▇▇█
train/global_step,▁▁▄▄▇▇██
train/grad_norm,█▁▃
